## PDF 解析演示 (Colab 兼容)

此 Notebook 演示了如何使用 `DocumentParser`（及相关组件）来解析 PDF 文档。代码已进行调整以便在 Google Colaboratory 环境中运行，特别是在处理路径和依赖项方面。

# study/demo_01_pdf_parsing.py
#
# 注意：此脚本具有重要的外部依赖项：
# 1. Docling 模型：需要通过 `Pipeline.download_docling_models()` 下载特定模型。
#    这些模型是核心解析功能所必需的。
# 2. Detectron2：`LayoutDetector` 组件（因此也包括 `DocumentParser`）
#    依赖于 Detectron2，其安装可能较为复杂，并可能有 GPU 要求。
# 3. 标准 Python 库：pdfminer.six、PyMuPDF (fitz)、Pillow。
#
# 成功运行此脚本需要一个预先配置好的、满足这些依赖项的环境。

In [ ]:
# === 安装必要的库 ===
# 如果您在 Google Colab 或本地环境中首次运行此 Notebook，可能需要安装以下依赖项。
# 请取消注释并运行此单元格中的相关行。

# 安装基础的 PDF 处理和图像库
# !pip install pdfminer.six pymupdf Pillow

# 安装 PyTorch (Detectron2 的依赖项)
# 请根据您的 CUDA 版本选择合适的 PyTorch 版本，或使用 CPU 版本。
# Colab 通常自带 PyTorch，但您可能需要特定版本。
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

# 安装 Detectron2
# Detectron2 的安装可能比较复杂，因为它涉及到编译。
# 以下是从源码安装的示例命令。您可能需要根据您的环境（CUDA 版本等）进行调整。
# 强烈建议参考 Detectron2 官方安装指南：https://detectron2.readthedocs.io/en/latest/tutorials/install.html
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# print("请确保在继续之前已成功安装所有依赖项，并根据需要重启运行时环境。")

### 依赖项说明

运行此 Notebook 需要几个关键的 Python 库和模型。

**主要依赖项包括：**

1.  **Docling 模型**: 这些是 `DocumentParser` 进行文档理解所需的核心模型。它们将通过代码自动下载（参见后续步骤）。
2.  **PDF 处理库**:
    *   `pdfminer.six`: 用于文本提取。
    *   `PyMuPDF` (fitz): 用于将 PDF 页面转换为图像等操作。
    *   `Pillow`: 用于图像处理。
3.  **Detectron2 与 PyTorch**:
    *   `torch`, `torchvision`: `Detectron2` 依赖这些 PyTorch 组件。
    *   `Detectron2`: `LayoutDetector` 组件依赖此库进行布局分析。Detectron2 的安装可能比较复杂，因为它可能需要针对您的特定 CUDA 环境进行编译。如果遇到问题，请务必查阅 [Detectron2 官方安装指南](https://detectron2.readthedocs.io/en/latest/tutorials/install.html)。

**安装说明:**

请参考上方 **代码单元格** 中的 `!pip install` 命令来安装这些库。您需要取消注释相关行并执行该单元格。对于 Detectron2，请特别注意其安装的复杂性，并优先参考官方文档。

**注意**: 在运行 `main()` 函数之前，确保所有必要的库都已成功安装。在 Colab 中，安装新库后可能需要 **重启运行时环境**。

### 导入标准库

导入必要的标准库。

In [ ]:
import sys
import os

### 设置 Python 路径

为了能够导入项目 `src` 目录下的自定义模块，我们需要将项目根目录添加到 Python 的搜索路径中。

In [ ]:
# 将 src 目录添加到 Python 路径以允许从 src 导入
# 在 Notebook 环境中 (如 Colab), __file__ 变量通常不可用。
# 因此，我们使用 os.getcwd() 获取当前工作目录 (即此 .ipynb 文件所在的 'study' 目录),
# 然后使用 os.path.join(..., '..') 来获取上级目录 (即项目根目录),
# 最后使用 os.path.abspath() 将其转换为绝对路径并添加到 sys.path。
# 这样，Python 解释器就能找到位于项目根目录下 src/ 中的模块。
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')))

### 导入自定义模块

从项目的 `src.pdf_parsing` 模块中导入核心的解析类。

In [ ]:
from src.pdf_parsing import DocumentParser, TextExtractor, LayoutDetector, StructureDetector, EntityRecognizer, Pipeline

### 定义主函数 `main()`

此函数封装了整个 PDF 解析的流程，包括模型下载、组件初始化、PDF 解析和结果输出。

In [ ]:
def main():
    # 下载 Docling 核心模型。
    # !!! 重要提示 !!!
    # 此步骤会从互联网下载模型文件，可能需要较长时间并消耗一定的网络数据。
    # 具体时间取决于您的网络连接速度以及模型是否已存在于缓存中。
    # 请确保您的网络连接稳定。
    print("Checking and downloading Docling models if necessary...")
    Pipeline.download_docling_models()
    print("Docling models are ready.")

    # 定义示例 PDF 文件的路径。此路径是相对于项目根目录的。
    #
    # === 在 Colab 中使用您自己的 PDF ===
    # 1. 上传文件:
    #    在 Colab 的左侧文件浏览器面板中，您可以直接上传 PDF 文件。
    #    上传后，文件通常位于 /content/your_file.name.pdf
    #    然后，您可以将 sample_pdf_path 修改为例如: "/content/your_file_name.pdf"
    # 2. 挂载 Google Drive:
    #    如果您想使用存储在 Google Drive 中的文件，可以挂载您的 Drive：
    #    from google.colab import drive
    #    drive.mount('/content/drive')
    #    然后将 sample_pdf_path 修改为指向您 Drive 中的文件，例如:
    #    sample_pdf_path = "/content/drive/MyDrive/path/to/your/document.pdf"
    #
    # 确保路径正确指向您要解析的 PDF 文件。
    sample_pdf_path = "data/test_set/pdf_reports/194000c9109c6fa628f1fed33b44ae4c2b8365f4.pdf"
    print(f"Using sample PDF: {sample_pdf_path}")

    # 检查示例 PDF 文件是否存在
    if not os.path.exists(sample_pdf_path):
        print(f"Error: Sample PDF file not found at {sample_pdf_path}")
        print("Please ensure the data is available in the 'data/test_set/pdf_reports/' directory or modify the path for Colab.")
        # 作为后备方案，让我们尝试列出目录中的文件以帮助调试
        print(f"Looking for data in: {os.path.abspath('data/test_set/pdf_reports/')}")
        if os.path.exists('data/test_set/pdf_reports/'):
             print(f"Files in 'data/test_set/pdf_reports/': {os.listdir('data/test_set/pdf_reports/')[:5]}") # 显示前 5 个
        else:
            print("'data/test_set/pdf_reports/' directory does not exist.")
        # 尝试检查 Colab 的 /content directory
        if os.path.exists('/content/'):
            print(f"Files in Colab '/content/': {os.listdir('/content/')[:10]}") # 显示前10个
        return

    # 初始化解析流程中使用的各个组件。
    # 在此演示中，我们将专注于文本提取和基本结构。
    # 更高级的组件（如 EntityRecognizer）可能需要特定的模型设置。
    text_extractor = TextExtractor()
    # 注意：LayoutDetector 依赖于 Detectron2 和相关模型。
    # Detectron2 的安装可能不简单。
    layout_detector = LayoutDetector() # 依赖于 Detectron2 和模型
    # StructureDetector 和 EntityRecognizer 对于简单演示来说可能设置过于复杂
    # 目前，让我们尝试使用 DocumentParser，它封装了其中的一些功能。

    print("Initializing DocumentParser...")
    # DocumentParser 可能会尝试加载所有模型，这可能非常耗资源。
    # 让我们看看是否可以仅使用文本和布局进行解析。
    # 我们可能需要根据 DocumentParser 的实际实现来调整这一点。
    # 初始化 DocumentParser。
    # 这是一个关键步骤，它会尝试加载所有相关的模型（文本、布局、结构等）。
    # 如果缺少依赖项（尤其是 Detectron2 或其模型），此步骤可能会失败。
    try:
        parser = DocumentParser() # 这可能会触发所有组件的模型加载
    except Exception as e:
        # DocumentParser 初始化失败时的后备逻辑。
        # 这通常表明存在环境配置问题或缺少必要的模型。
        print(f"Error initializing DocumentParser: {e}")
        print("This might be due to missing models or dependencies for all components. Please check installation steps.")
        print("Attempting a more basic parsing approach...")
        # 后备方案：如果 DocumentParser 对于快速演示来说过于复杂，请尝试直接使用 TextExtractor
        try:
            doc_bytes = open(sample_pdf_path, "rb").read()
            # TextExtractor 通常在布局检测后逐页工作。
            # 如果 DocumentParser 有问题，让我们尝试模拟一个简化的流程。
            # 这部分可能需要根据实际的类接口进行调整。
            # 对于真正的演示，我们理想情况下会使用完整的 Pipeline 或 DocumentParser。
            # 如果需要直接进行文本提取，过程会更复杂。
            print("DocumentParser initialization failed. A full demo of DocumentParser requires all models.")
            print("For a simplified text extraction, you would typically integrate TextExtractor within a pipeline.")
            print("This demo will focus on what can be achieved with available components.")
            # 作为替代方案，我们可以尝试显示 pdfminer.six 检测到多少页。
            from pdfminer.high_level import extract_pages
            try:
                page_count = 0
                for _ in extract_pages(sample_pdf_path):
                    page_count +=1
                print(f"Basic check: pdfminer.six detected {page_count} pages in the document.")
            except Exception as pe:
                print(f"Error during basic pdfminer.six check: {pe}")

            return # 如果 DocumentParser 失败则退出，因为它是预期演示的关键
        except Exception as fallback_e:
            print(f"Error in fallback parsing attempt: {fallback_e}")
            return


    # 解析 PDF 文档
    # 使用配置好的 DocumentParser 解析指定的 PDF 文件。
    # 此过程包括页面分割、图像转换、布局检测、文本提取等。
    print(f"Parsing PDF: {sample_pdf_path}")
    try:
        # `parse` 方法应接受 PDF 路径并返回一个 Document 对象
        document = parser.parse(sample_pdf_path)
    except Exception as e:
        # PDF 解析过程中发生错误。
        # 这可能由多种原因引起，例如 PDF 文件损坏、模型不兼容或特定页面内容导致的问题。
        print(f"Error during PDF parsing: {e}")
        print("This could be due to issues with the PDF file or model incompatibilities.")
        print("Ensure all models for DocumentParser components are correctly downloaded and configured.")
        # 让我们尝试看看是否是某个特定组件导致了问题
        # 这是用于在上述操作失败时的调试目的。
        print("Attempting to extract text directly to see if that part works...")
        try:
            from src.pdf_parsing.utils import pdf_to_images_pymupdf
            from PIL import Image
            doc_images = list(pdf_to_images_pymupdf(sample_pdf_path))
            if not doc_images:
                print("Could not convert PDF to images using PyMuPDF.")
                return
            
            first_page_image = Image.open(doc_images[0])
            extracted_text_elements = text_extractor.extract_text(first_page_image, page_number=0)
            
            if extracted_text_elements:
                print(f"Direct text extraction from first page (first 5 elements): {extracted_text_elements[:5]}")
                first_page_text_content = " ".join([te.text for te in extracted_text_elements])
                print("\n--- Text Content of First Page (Direct Extraction) ---")
                print(first_page_text_content[:1000]) # 打印前 1000 个字符
                print("----------------------------------------------------")
            else:
                print("Direct text extraction yielded no elements.")

        except Exception as te_e:
            print(f"Error during direct text extraction attempt: {te_e}")
        return

    # 解析成功后，获取并打印文档的总页数。
    # 打印已解析的页数
    num_pages = len(document.pages)
    print(f"\nSuccessfully parsed document. Number of pages: {num_pages}")

    # 提取并打印第一页的文本内容，以进行快速验证。
    # 注意：根据 Document 对象结构，文本内容可能存储在不同的属性中。
    # 打印第一页的文本内容
    if num_pages > 0:
        first_page = document.pages[0]
        print("\n--- Text Content of First Page ---")
        # Page 对象的 'text' 属性应提供其完整的文本内容
        # 或者，它可能是一个需要连接起来的文本块列表。
        # 这取决于 pdf_parsing.py 中 Page 对象的结构
        if hasattr(first_page, 'text_content'): # 假设有一个 'text_content' 字段
            print(first_page.text_content[:1000]) # 打印前 1000 个字符
        elif hasattr(first_page, 'text_blocks') and first_page.text_blocks: # 假设它有 text_blocks
            full_text = " ".join([block.text for block in first_page.text_blocks if hasattr(block, 'text')])
            print(full_text[:1000]) # 打印前 1000 个字符
        elif hasattr(first_page, 'text_elements') and first_page.text_elements: # 常见的替代方案
            full_text = " ".join([elem.text for elem in first_page.text_elements if hasattr(elem, 'text')])
            print(full_text[:1000])
        else:
            print("Could not find a direct text attribute on the first page object.")
            print("Page object details:", dir(first_page)) # 以了解其结构
        print("----------------------------------")
    else:
        print("The document has no pages or pages could not be parsed.")

### 执行主程序

调用 `main()` 函数来运行整个 PDF 解析流程。

In [ ]:
main()